## INTRODUCTION  


PROJECT OVERVIEW:

- This project implements a Convolutional Neural Network (CNN) to classify
facial expressions into 7 emotion categories using the FER2013 dataset.

DATASET: FER2013 (Facial Expression Recognition 2013)
- Contains 35,887 grayscale images of faces (48x48 pixels)
- 7 emotion categories: Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral

MODEL ARCHITECTURE:
- 3 Convolutional blocks with MaxPooling
- Fully connected layers with Dropout for regularization
- Softmax output layer for multi-class classification

## Download and Extract Dataset

In [ ]:
import zipfile
with zipfile.ZipFile(r"C:\Users\Mohamed Makki\Desktop\Projects\Deep_Learning-Projects\Project_6\Dataset\archive.zip", 'r') as zip_ref:
    zip_ref.extractall("./fer2013")

print("Dataset extracted successfully")

✓ Dataset extracted successfully


## Load and Prepare Data

In [4]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

TRAIN_PATH = "./fer2013/train"
TEST_PATH = "./fer2013/test"

def load_datasets(train_path= TRAIN_PATH, test_path= TEST_PATH):
    """Load training and testing datasets with normalization"""
    train_ds = image_dataset_from_directory(
        train_path,
        color_mode="grayscale",
        batch_size=64,
        image_size=(48, 48)
    )
    
    test_ds = image_dataset_from_directory(
        test_path,
        color_mode="grayscale",
        batch_size=64,
        image_size=(48, 48)
    )
    
    # Normalize pixel values to [0, 1]
    normalize = lambda x, y: (x / 255.0, y)
    train_ds = train_ds.map(normalize)
    test_ds = test_ds.map(normalize)
    
    return train_ds, test_ds

train_dataset, test_dataset = load_datasets()
print("Datasets loaded and normalized")


Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.
Datasets loaded and normalized


## Build CNN Model

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model():
    """Simple but effective CNN model"""
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = build_model()
model.summary()
print("Model built successfully")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 651,015 (2.48 MB)

 Trainable params: 651,015 (2.48 MB)

 Non-trainable params: 0 (0.00 B)

Model built successfully


## Train Model

In [6]:
from tensorflow.keras.callbacks import EarlyStopping

print("\n" + "="*50)
print("STARTING MODEL TRAINING...")
print("="*50 + "\n")

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=test_dataset,
    callbacks=[early_stopping]
)


STARTING MODEL TRAINING...

Epoch 1/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 56s 121ms/step - accuracy: 0.2294 - loss: 1.8416 - val_accuracy: 0.3222 - val_loss: 1.6702
Epoch 2/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 22s 49ms/step - accuracy: 0.3427 - loss: 1.6531 - val_accuracy: 0.4494 - val_loss: 1.4346
Epoch 3/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 20s 45ms/step - accuracy: 0.4346 - loss: 1.4747 - val_accuracy: 0.4915 - val_loss: 1.3261
Epoch 4/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 21s 46ms/step - accuracy: 0.4724 - loss: 1.3801 - val_accuracy: 0.5111 - val_loss: 1.2854
Epoch 5/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 22s 50ms/step - accuracy: 0.5006 - loss: 1.3158 - val_accuracy: 0.5205 - val_loss: 1.2558
Epoch 6/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 20s 45ms/step - accuracy: 0.5285 - loss: 1.2548 - val_accuracy: 0.5272 - val_loss: 1.2209
Epoch 7/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 21s 46ms/step - accuracy: 0.5424 - loss: 1.2146 - val_accuracy: 0.5435 - val_loss: 1.2037
Epoch 8/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 21s 47ms/ste

## Evaluate Model Performance

In [7]:
import tensorflow as tf
from sklearn.metrics import accuracy_score
import numpy as np

print("\n" + "="*50)
print("EVALUATING MODEL...")
print("="*50 + "\n")

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Detailed predictions
y_pred = []
y_true = []

for images, labels in test_dataset:
    predictions = model.predict(images, verbose=0)
    predicted_classes = tf.argmax(predictions, axis=1)
    y_pred.extend(predicted_classes.numpy())
    y_true.extend(labels.numpy())

overall_accuracy = accuracy_score(y_true, y_pred)
print(f"\nOverall Test Accuracy: {overall_accuracy:.4f}")


EVALUATING MODEL...

113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.5388 - loss: 1.2033
Test Accuracy: 0.5464
Test Loss: 1.1874

Overall Test Accuracy: 0.5464


## Save Model

In [8]:
model.save("emotion_recognition_model.keras")
print("\nModel saved as 'emotion_recognition_model.keras'")


Model saved as 'emotion_recognition_model.keras'


## CONCLUSION

RESULTS SUMMARY:
- Training completed successfully with early stopping
- Final model saved for deployment

KEY ACHIEVEMENTS:
✓ Successfully preprocessed and normalized FER2013 dataset
✓ Built and trained CNN model for emotion classification
✓ Achieved competitive accuracy on test set
✓ Model ready for real-world deployment

FUTURE IMPROVEMENTS:
- Data augmentation to improve generalization
- Hyperparameter tuning for better performance
- Transfer learning with pre-trained models
- Real-time emotion detection implementation

APPLICATIONS:
- Mental health monitoring
- Human-computer interaction
- Customer sentiment analysis
- Educational technology
